# 循环神经网络RNN

RNN是一种主要用于处理时序数据的深度模型. 其常涉及的工作如:

- **语音识别(Speech recognition)**: 给一段讲话的语音,将其转化为文字表述(多对多);
- **音乐生成(Music generation)**: 给一个音符, 生成一段音乐(一对多);
- **情感分类(Sentiment classification)**: 给一段某商品下用户的留言, 为用户留言的情感分类(消极或者积极)(多对一);
- **机器翻译(Machine translation)**: 将一种语言翻译成另一种语言(多对多);
- **视频运动分析(Video activity recognition)**: 给一段视频,然后要求你识别其中的行为(多对一);
- **命名实体识别(Name entity recognition)**: 给定一个句子要你识别出句中的人名(多对多);

可以发现,上面问题的输入$X$输出$Y$大部分都是**时序序列**, 这是RNN可处理问题的一大特征.

下面将从命名实体识别问题探讨如何从0开始建立一个RNN网络。

## 命名实体识别（Name entity recognition）

模型想处理的问题是找出一句话中的人名，比如输入$X$是：**"Harry Potter and Herminoe Granger invented a new spell."** (这些人名都是出自于J.K.Rowling笔下的系列小说Harry Potter)，输出$Y$也是一个序列，其表示了每一个单词是否是人名的一部分。我们用下面的图表示模型：

![rnn1](./resources/rnn1.png)

- 我们使用$x^{<1>}, x^{<2>}, ..., x^{<t>},..., x^{<9>}$表示输入的每一个单词，他们是一个**时序序列**，会按照时间先后顺序输入模型。另一方面，使用$T_x=9$表示输入序列的时间长度。
- 同样$y^{<1>}, y^{<2>}, ..., y^{<t>},..., y^{<9>}$表示每一个时刻模型的输出。使用$T_y=9$表示输出序列的时间长度。
- 针对多组训练数据，使用$x^{(i)<t>}$表示第$i$组训练数据的第$t$时刻的输入；使用$y^{(i)<t>}$表示第$i$组训练数据的第$t$时刻的输出；使用$T_x^{(i)}$表示第$i$组训练数据输入序列的时间长度；使用$T_y^{(i)}$表示第$i$组训练数据输出序列的时间长度。

### 如何表征输入？

上面模型每一个时刻的输入是单词，单词不能直接输入模型，需要转换成某种统一的表示后网络才能处理。最常用的表征方式如**one-hot方式**。

one-hot即先排列出数据集中所有出现的词，然后表征每一个词用该位置1，其他位置0，这样生成的一个稀疏的one-hot向量来表示(还有一些特殊处理如：Unknown word的处理)。

### 使用标准神经网络

针对上面的命名实体识别，可以尝试的方法之一就是使用**标准神经网络**，其建立的网络结构可如下：

![rnn2](./resources/rnn2.png)

但是结果表明这个方法并不好，主要存在的问题可能在于：

- **输入序列和输出序列在不同例子中可以有不同的长度**，即使可以设置最大长度填充，但仍然不是较好的表示方法。
- **不共享从文本的不同位置上学到的特征**，即句子前面部分学习到的信息不能在后面部分使用。比如前面已经学习到**Harry**是人名的一部分，句子后面部分又出现了这个词可能并无法被识别出来。
- **参数巨量**，使用One-hot表示的句子作为上面网络的输入，第一层会有海量的参数。

### 循环神经网络

下面介绍另一种处理这种时序数据的模型：循环神经网络。它的结构如下：

![rnn3](./resources/rnn3.png)

从左往右依次读入句中每一个单词，将第一个单词$x^{<1>}$输入网络，然后尝试预测输出$y^{<1>}$，接着输入$x^{<2>}$来预测$y^{<2>}$，但不一样的地方在于，预测$y^{<2>}$的时候**会使用前一时刻产生的一个激活值$a^{<1>}$作为输入**，这样顺序进行直到输入结束。

需要注意的时候，上面1图是将RNN在时间上展开了，真正的结构更像2图所示，一个RNN是共用**一种结构和一套权值（$W_{ax}, W_{aa}, W_{ya}$）**，且一般我们构造$a^{<0>}$为全0向量。

另一点需要注意的是，上面神经网络结构的一个限制是它在某一时刻的预测**仅使用了序列之前的输入信息而并没有使用序列中后部分的信息**。

假设我们将第一个时刻的RNN内部结构展开，可得到如下结构图：

![rnn4](./resources/rnn4.jpeg)

按照上图所示，可得到如下计算式子：

> $a^{<1>}=g_1(W_{aa}a^{<0>}+W_{ax}x^{<1>}+b_a)$  
> $\hat{y}^{<1>}=g_2(W_{ya}a^{<1>}+b_y)$

一般$g_1$常使用**tanh**或**ReLU**，$g_2$则取决于输出，如二分类就可能用sigmoid，多分类可能用softmax。**要注意的是上面的$W_{ax}, W_{aa}, W_{ya}, b_a, b_y$是整个网络公用的一套参数，它在每一个时刻都被使用，需要训练的也是这些参数**。


假设$x^{<1>}$的维度为500, $a^{<0>}$与$a^{<1>}$的维度都为100，下面列出了简化后的符号表示：

> $a^{<1>}=g_1(W_{a}[a^{<0>},x^{<1>}]+b_a)$  
> $\hat{y}^{<1>}=g_2(W_{y}a^{<1>}+b_y)$

$W_a$的维度为$(500+100, 100)$, 输入$[a^{<0>},x^{<1>}]$的维度变为$(500+100, 1)$。